In [1]:
import numpy as np
import librosa
from panns_inference import AudioTagging
model = AudioTagging(checkpoint_path=None, device='cpu')  # Use 'cpu' if GPU is not available


Checkpoint path: /Users/A117792292/panns_data/Cnn14_mAP=0.431.pth
Using CPU.


In [2]:
#TODO spravit 8 sekundove embedingy
def select_strongest_8_seconds(y, sr, onset_env, beats):
    # Calculate the cumulative onset strength for each 8-second segment
    hop_length = 512
    frame_length = int(8 * sr / hop_length)
    onset_env_cumsum = np.cumsum(onset_env)
    max_strength = 0
    start_frame = 0

    for i in range(len(onset_env) - frame_length):
        strength = onset_env_cumsum[i + frame_length] - onset_env_cumsum[i]
        if strength > max_strength:
            max_strength = strength
            start_frame = i

    # Align the start of the 8-second segment with the nearest beat
    start_time = librosa.frames_to_time(start_frame, sr=sr, hop_length=hop_length)
    beat_times = librosa.frames_to_time(beats, sr=sr, hop_length=hop_length)

    if len(beat_times) > 0:
        nearest_beat_time = min(beat_times, key=lambda x: abs(x - start_time))
        start_time = nearest_beat_time

    end_time = start_time + 8
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    y_selected_seconds = y[start_sample:end_sample]
    return y_selected_seconds, start_time, end_time

In [3]:
# Function to normalize a vector
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

# Function to get an embedding of an audio file
def get_embedding(file_path):
    # Load the audio file
    # audio, _ = librosa.load(audio_file, sr=None)
    y, sr = librosa.load(file_path, sr=None)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr, aggregate=np.median)
    tempo, beats = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr, trim=True)
    y_selected_seconds, start_time, end_time = select_strongest_8_seconds(y, sr, onset_env, beats)
    query_audio = y_selected_seconds[None, :]  # Reshape for model input
    # Load the PANNS model with the correct checkpoint path

    # Perform inference to get the embedding
    _, emb = model.inference(query_audio)

    # Normalize the embedding
    normalized_emb = normalize(emb[0])
    return normalized_emb


# # Example usage
audio_file = '../mp3songs/tekkno/165Er Zahnfleischbluten.mp3'
embedding = get_embedding(audio_file)

In [4]:
embedding.shape

(2048,)

In [5]:
import pandas as pd
df = pd.read_csv('test_set_final_13main_59sub_genres.csv')
display(df)

,name,artist,spotify_id,preview,img,danceability,energy,loudness,speechiness,acousticness,...,artist_id,genres,genres_str,main_genre,file_path,genre_array2,VGG_embeddings_8_seconds_base_13_genres_test,fine_tuned_13_genres_with_val_softmax_vggish_model_batch_size_5_test,fine_tuned_59_genres_with_val_sigmoid_vggish_model_multilabel_batch_size_5,base_open3l_model_test_data
0,Brown Noise 140 hz Sharp Q,Granular,1efbB8dNowPNwBU6HaHjup,https://p.scdn.co/mp3-preview/587cffc089799746...,https://i.scdn.co/image/ab67616d0000b273727265...,0.205,0.337,-37.221,0.0647,0.75100,...,1jfnMuDBl5OaAoU0VwLD8m,['sleep'],['sleep'],ambient,../mp3songs/48_genres_11_main_genres/Brown Noi...,[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,"[-0.20768842101097107, 0.17027533054351807, 0....","[-1.7131984233856201, 1.8003138303756714, 0.26...","[-14.115251541137695, -7.341050148010254, -2.3...",[ 2.0042045 3.385865 2.2807853 0.486558...
1,Survival Tactics,Joey Bada$$,4pNT8XTKWb2vhcsqwzdXCT,https://p.scdn.co/mp3-preview/89f912f9bbd2af71...,https://i.scdn.co/image/ab67616d0000b273fcd372...,0.530,0.843,-2.271,0.3900,0.42800,...,2P5sC9cVZDToPxyomzF1UH,"['hip hop', 'rap']","['hip hop', 'rap']",rap,../mp3songs/48_genres_11_main_genres/Survival ...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,"[-0.5984508991241455, -0.21273860335350037, 0....","[-4.477705478668213, -2.5627753734588623, -2.5...","[-4.634706020355225, -2.2025468349456787, -2.5...",[2.7665234 2.2206402 2.4165297 2.279251 1...
2,That Day Will Come,Capleton,6D4eMFcg3DquhNOQsNanQk,https://p.scdn.co/mp3-preview/d33eccef67aaf6ac...,https://i.scdn.co/image/ab67616d0000b27352630f...,0.672,0.839,-5.034,0.1140,0.06740,...,21J3YJTyq1biE3SvSNjzuf,"['reggae', 'roots reggae']","['reggae', 'roots reggae']",reggae,../mp3songs/48_genres_11_main_genres/That Day ...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,"[-0.6541938781738281, 0.03160589933395386, 0.2...","[-4.383584022521973, -2.044909715652466, -1.62...","[-5.128568172454834, -2.1408400535583496, -2.6...",[2.2732286 2.584271 3.2119744 2.348352 1...
3,Already Gone,Kelly Clarkson,4fq2YUONcgrCJ2SPndSeKC,https://p.scdn.co/mp3-preview/660bd02563cebe61...,https://i.scdn.co/image/ab67616d0000b2737ed879...,0.209,0.872,-2.996,0.0757,0.21700,...,3BmGtnKgCSGYIUhmivXKWX,"['dance pop', 'pop']","['dance pop', 'pop']",pop,../mp3songs/48_genres_11_main_genres/Already G...,[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,"[-0.4340738356113434, -0.13108360767364502, 0....","[-3.3959813117980957, -1.1046748161315918, -1....","[-5.52520751953125, -1.0195895433425903, -0.75...",[2.3422306 2.3426332 2.685641 2.38519 2...
4,The Drugs,The Holdup,6CTxqx9pxaoOxOYZSbsqDc,https://p.scdn.co/mp3-preview/c544ea6c0d9720a9...,https://i.scdn.co/image/ab67616d0000b273a580ae...,0.723,0.810,-3.983,0.0338,0.03370,...,1gwTohHbTpE3qdPyDvnOEu,['reggae fusion'],['reggae fusion'],reggae,../mp3songs/48_genres_11_main_genres/The Drugs...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0...,"[-1.0772324800491333, 0.011838555335998535, 0....","[-6.385115623474121, -1.9373447895050049, -2.0...","[-6.066783905029297, -1.9855862855911255, -1.5...",[2.1347637 2.1387303 3.246513 2.245867 1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,"The Good, the Bad and the Dirty",Panic! At The Disco,5eoZTdiq4C4aoEfUwAN0ha,https://p.scdn.co/mp3-preview/f37091eef36c052b...,https://i.scdn.co/image/ab67616d0000b27323152d...,0.531,0.825,-5.333,0.0545,0.00191,...,20JZFwl6HVl6yg8a4H3ZqK,['pop'],['pop'],pop,"../mp3songs/48_genres_11_main_genres/The Good,...",[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,"[-0.633353590965271, -0.1372760832309723, 0.10...","[-4.58098030090332, -2.0702669620513916, -2.02...","[-5.9073100090026855, -2.2739641666412354, -3....",[2.212627 2.329846 2.4827034 2.4413834 1...
840,The End Is Where We Begin,Thousand Foot Krutch,1Ou0kPQcfXuLVYUKV8KUox,https://p.scdn.co/mp3-preview/5037771d0b7498bc...,https://i.scdn.co/image/ab67616d0000b273405821.

In [14]:
# len(embedding)

2048

In [15]:
# embedding

array([0.        , 0.        , 0.        , ..., 0.03252791, 0.        ,
       0.        ], dtype=float32)

In [16]:
import pandas as pd
df = pd.read_csv('spotify_songs_20k_snapshot_filtered_limited_with_paths.csv')
desired_genres = ['pop', 'hip hop', 'r&b', 'edm', 'classic rock', 'country']

# Filter the DataFrame to include only rows with the desired first genres
filtered_df = df[df['first_genre'].isin(desired_genres)]

# Select only 10 rows of each genre
filtered_df = filtered_df.groupby('first_genre').head(10)

# Display the filtered DataFrame
display(filtered_df)

,name,artist,spotify_id,preview,img,danceability,energy,loudness,speechiness,acousticness,...,danceability_artist,energy_artist,instrumentalness_artist,liveness_artist,speechiness_artist,valence_artist,artist_id,genres,first_genre,file_path
4,you broke me first,Tate McRae,45bE4HXI0AwGZXfZtMp8JR,https://p.scdn.co/mp3-preview/47afb10323067e82...,https://i.scdn.co/image/ab67616d0000b2730c2c97...,0.642,0.374,-9.386,0.0545,0.78600,...,0.617000,0.410176,2.558588e-05,0.172071,0.066106,0.309882,45dkTj5sMRSjrmBSBeiHym,['pop'],pop,../mp3songs/spotify_dataset/you broke me first...
5,POPSTAR (feat. Drake),DJ Khaled,6EDO9iiTtwNv6waLwa1UUq,https://p.scdn.co/mp3-preview/f06dde2517250354...,https://i.scdn.co/image/ab67616d0000b273efaecb...,0.800,0.560,-4.818,0.2610,0.05700,...,0.634541,0.753162,3.464865e-07,0.297568,0.159468,0.482432,0QHgL1lAIqAw0HtD7YldmP,"['hip hop', 'miami hip hop', 'pop rap', 'rap']",hip hop,../mp3songs/spotify_dataset/POPSTAR (feat. Dra...
6,Go Crazy,Chris Brown,1IIKrJVP1C9N7iPtG6eOsK,https://p.scdn.co/mp3-preview/cd1e4584425d4be8...,https://i.scdn.co/image/ab67616d0000b27363e0dd...,0.755,0.578,-8.932,0.1450,0.31600,...,0.660851,0.640302,7.828409e-05,0.172564,0.130031,0.486374,7bXgB6jMjp9ATFy66eO08Z,"['r&b', 'rap']",r&b,../mp3songs/spotify_dataset/Go Crazy.mp3
7,Midnight Sky,Miley Cyrus,27ycaQnQAxaPiyeg3nr2aB,https://p.scdn.co/mp3-preview/837e5649e915ca76...,https://i.scdn.co/image/ab67616d0000b273d59c3c...,0.705,0.814,-5.466,0.0346,0.00115,...,0.568616,0.735849,8.239315e-05,0.170552,0.063982,0.447863,5YGY8feqx7naU7z4HrwZM6,['pop'],pop,../mp3songs/spotify_dataset/Midnight Sky.mp3
9,GREECE (feat. Drake),DJ Khaled,35RJhm1pEovTBwnNR0zWad,https://p.scdn.co/mp3-preview/730e8f890f342ab3...,https://i.scdn.co/image/ab67616d0000b2737db0d2...,0.695,0.343,-10.798,0.0432,0.07430,...,0.634541,0.753162,3.464865e-07,0.297568,0.159468,0.482432,0QHgL1lAIqAw0HtD7YldmP,"['hip hop', 'miami hip hop', 'pop rap', 'rap']",hip hop,../mp3songs/spotify_dataset/GREECE (feat. Drak...
15,Adore You,Harry Styles,3jjujdWJ72nww5eGnfs2E7,https://p.scdn.co/mp3-preview/2dbeaa922270b337...,https://i.scdn.co/image/ab67616d0000b27377fdcf...,0.676,0.771,-3.675,0.0483,0.02370,...,0.541462,0.586769,1.680888e-02,0.156208,0.035888,0.406896,6KImCVD70vtIoJWnq6nGn3,['pop'],pop,../mp3songs/spotify_dataset/Adore You.mp3
17,Falling,Harry Styles,1ZMiCix7XSAbfAJlEZWMCp,https://p.scdn.co/mp3-preview/755e88e49f038885...,https://i.scdn.co/image/ab67616d0000b27377fdcf...,0.567,0.267,-6.502,0.0299,0.83900,...,0.541462,0.586769,1.680888e-02,0.156208,0.035888,0.406896,6KImCVD70vtIoJWnq6nGn3,['pop'],pop,../mp3songs/spotify_dataset/Falling.mp3
21,Perfect,Ed Sheeran,0tgVpDi06FyKpA1z0VMD4v,https://p.scdn.co/mp3-preview/9779493d90a47f29...,https://i.scdn.co/image/ab67616d0000b273ba5db4...,0.599,0.448,-6.312,0.0232,0.16300,...,0.660442,0.503588,2.381436e-04,0.176192,0.077594,0.508474,6eUKZXaKkcviH0Ku9w2n3V,"['pop', 'singer-songwriter pop', 'uk pop']",pop,../mp3songs/spotify_dataset/Perfect.mp3
27,Say You Won't Let Go,James Arthur,5uCax9HTNlzGybIStD3vDh,https://p.scdn.co/mp3-preview/00825cb1779b31d6...,https://i.scdn.co/image/ab67616d0000b27320beb6...,0.358,0.557,-7.398,0.0590,0.69500,...,0.453781,0.548656,3.431875e-06,0.135084,0.054506,0.335281,4IWBUUAFIplrNtaOHcJPRM,"['pop', 'talent show', 'uk pop']",pop,../mp3songs/spotify_dataset/Say You Won't Let ...
34,Shape of You,Ed Sheeran,7qiZfU4dY1lWllzX7mPBI3,https://p.scdn.co/mp3-preview/84462d8e1e4d0f9e...,https://i.scdn.co/image/ab67616d0000b273ba5db4...,0.825,0.652,-3.183,0.0802,0.58100,...,0.660442,0.503588,2.381436e-04,0.176192,0.077594,0.508474,6eUKZXaKkcviH0Ku9w2n3V,"['pop', 'singer-songwriter pop', 'uk pop']",pop,../mp3songs/spotify_dataset/Shape of You.mp3


In [6]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
import pandas as pd
from tqdm import tqdm

# Initialize Qdrant client
client = QdrantClient("http://localhost:6333")
csv_file = 'test_set_final_13main_59sub_genres.csv'

# Initialize the DataFrame
df = pd.read_csv(csv_file)

# Initialize the new column to None
df['base_PANNs_transformer_model_test_data'] = None

batch_size = 10
embeddings = []

# Global ID counter
global_id = 0

# Iterate over each row in the DataFrame with progress tracking
for i, file_path in enumerate(tqdm(df['file_path'])):
    embedding = get_embedding(file_path)
    embeddings.append(embedding)

    # Assign the embedding to the DataFrame
    df.at[i, 'base_PANNs_transformer_model_test_data'] = embedding

    # If batch size is reached, upsert the batch and reset embeddings list
    if (i + 1) % batch_size == 0 or (i + 1) == len(df):
        points = [
            PointStruct(id=global_id + index, vector=embedding.flatten(), payload=row[1].drop(['base_PANNs_transformer_model_test_data','base_open3l_model_test_data', 'VGG_embeddings_8_seconds_base_13_genres_test','fine_tuned_13_genres_with_val_softmax_vggish_model_batch_size_5_test','fine_tuned_59_genres_with_val_sigmoid_vggish_model_multilabel_batch_size_5']).to_dict())
            for index, (embedding, row) in enumerate(zip(embeddings, df.iloc[i + 1 - batch_size:i + 1].iterrows()))
        ]

        operation_info = client.upsert(
            collection_name="base_PANNs_transformer_model_test_data",
            wait=True,
            points=points,
        )
        print(f"Upserted batch {(i + 1) // batch_size} from: {csv_file}")
        print(operation_info)

        # Update global ID counter
        global_id += len(points)

        # Reset embeddings list
        embeddings = []
        df.to_csv(csv_file, index=False)
        print(f"Saved DataFrame with embeddings to: {csv_file}")

# Save the DataFrame with embeddings
df.to_csv(csv_file, index=False)
print(f"Saved DataFrame with embeddings to: {csv_file}")

  1%|          | 9/844 [00:06<09:49,  1.42it/s]

Upserted batch 1 from: test_set_final_13main_59sub_genres.csv
operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


  1%|          | 10/844 [00:08<16:29,  1.19s/it]

Saved DataFrame with embeddings to: test_set_final_13main_59sub_genres.csv


  2%|▏         | 19/844 [00:15<10:07,  1.36it/s]

...

100%|█████████▉| 843/844 [17:27<00:01,  1.17s/it]

Upserted batch 84 from: test_set_final_13main_59sub_genres.csv
operation_id=84 status=<UpdateStatus.COMPLETED: 'completed'>


100%|██████████| 844/844 [17:30<00:00,  1.24s/it]

Saved DataFrame with embeddings to: test_set_final_13main_59sub_genres.csv


Saved DataFrame with embeddings to: test_set_final_13main_59sub_genres.csv


In [2]:
import os
import requests

# Create a directory for PANNS data
panns_data_dir = '/Users/A117792292/panns_data'
os.makedirs(panns_data_dir, exist_ok=True)

# URLs of the required files
files_to_download = {
    'class_labels_indices.csv': 'https://raw.githubusercontent.com/qiuqiangkong/audioset_tagging_cnn/master/panns_data/class_labels_indices.csv'
}

# Download the files
for filename, url in files_to_download.items():
    response = requests.get(url)
    with open(os.path.join(panns_data_dir, filename), 'wb') as f:
        f.write(response.content)

print("Required files downloaded and saved.")

Required files downloaded and saved.


In [4]:
import os
import requests

# Create a directory for PANNS data
panns_data_dir = '/Users/A117792292/panns_data'
os.makedirs(panns_data_dir, exist_ok=True)

# URL of the required model file
model_url = 'https://zenodo.org/record/3987831/files/Cnn14_mAP%3D0.431.pth?download=1'
model_filename = 'Cnn14_mAP=0.431.pth'

# Download the model file
response = requests.get(model_url)
with open(os.path.join(panns_data_dir, model_filename), 'wb') as f:
    f.write(response.content)

print("Required model file downloaded and saved.")

Required model file downloaded and saved.


In [ ]:
import torch
from panns_inference import AudioTagging, Cnn14

# Define the model and load the checkpoint
checkpoint_path = '/Users/A117792292/panns_data/Cnn14_mAP=0.431.pth'
model = Cnn14()
checkpoint = torch.load(checkpoint_path, map_location='cpu')

# Remove the fc_audioset layer from the checkpoint if it causes a size mismatch
if 'fc_audioset.weight' in checkpoint:
    del checkpoint['fc_audioset.weight']
if 'fc_audioset.bias' in checkpoint:
    del checkpoint['fc_audioset.bias']

# Load the state dictionary into the model
model.load_state_dict(checkpoint, strict=False)

# Initialize the AudioTagging class with the loaded model
audio_tagging = AudioTagging(model=model, device='cpu')

# Example usage
audio_file = '../mp3songs/tekkno/165Er Zahnfleischbluten.mp3'
embedding = get_embedding(audio_file)
print(embedding)